Copyright 2020 Konstantin Yakovlev, Matthias Anderer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)


In [3]:
LOSS_MULTIPLIER = 0.95 # Set multiplier according to desired under-/overshooting

In [4]:
# define custom loss function
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * LOSS_MULTIPLIER)
    hess = np.where(residual < 0, 2, 2 * LOSS_MULTIPLIER)
    return grad, hess

# define custom evaluation metric
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * LOSS_MULTIPLIER) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [5]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]
    
    ############
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


In [6]:
########################### Model params
#################################################################################
import lightgbm as lgb
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric':'rmse',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5
}



In [12]:
########################### Vars
#################################################################################
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913+28            # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = False               # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]

#PATHS for Features
ORIGINAL = 'C://Users//nkyam//Desktop//m5_forecast//'
BASE     = 'C://Users//nkyam//Desktop//m5_forecast//grid_part_1.pkl'
PRICE    = 'C://Users//nkyam//Desktop//m5_forecast//grid_part_2.pkl'
CALENDAR = 'C://Users//nkyam//Desktop//m5_forecast//grid_part_3.pkl'

#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL+'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
STORES_IDS 


['CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX_1',
 'TX_2',
 'TX_3',
 'WI_1',
 'WI_2',
 'WI_3']

In [13]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          num_boost_round = 3600, 
                          early_stopping_rounds = 50, 
                          valid_sets = [train_data, valid_data],
                          verbose_eval = 100,
                          fobj = custom_asymmetric_train

                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
[LightGBM] [Info] Saving data to binary file train_data.bin
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.130598 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.57235	valid_1's rmse: 2.23027
[200]	training's rmse: 2.47537	valid_1's rmse: 2.15452
[300]	training's rmse: 2.41545	valid_1's rmse: 2.11561
[400]	training's rmse: 2.3721	valid_1's rmse: 2.08599
[500]	training's rmse: 2.33791	valid_1's rmse: 2.06401
[600]	training's rmse: 2.30512	valid_1's rmse: 2.04144
[700]	tr

'rm' is not recognized as an internal or external command,
operable program or batch file.


Train CA_2
[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.126853 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.57235	valid_1's rmse: 2.42549
Early stopping, best iteration is:
[84]	training's rmse: 2.59645	valid_1's rmse: 2.4206
Train CA_3


'rm' is not recognized as an internal or external command,
operable program or batch file.


[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.117228 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.57235	valid_1's rmse: 3.49603
[200]	training's rmse: 2.47537	valid_1's rmse: 3.47815
Early stopping, best iteration is:
[247]	training's rmse: 2.44393	valid_1's rmse: 3.46948
Train CA_4


...

'rm' is not recognized as an internal or external command,
operable program or batch file.


[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.123459 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[6]	training's rmse: 3.13456	valid_1's rmse: 4.55872
Train WI_3


'rm' is not recognized as an internal or external command,
operable program or batch file.


[LightGBM] [Warning] File train_data.bin exists, cannot save binary to it
[LightGBM] [Info] Load from binary file train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.124784 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.57235	valid_1's rmse: 2.87935
Early stopping, best iteration is:
[84]	training's rmse: 2.59645	valid_1's rmse: 2.87731


'rm' is not recognized as an internal or external command,
operable program or batch file.


In [14]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        if USE_AUX:
            model_path = AUX_MODELS + model_path
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds

Predict | Day: 1
##########  0.03 min round |  0.03 min total |  35741.28 day sales |
Predict | Day: 2
##########  0.03 min round |  0.07 min total |  32934.18 day sales |
Predict | Day: 3
##########  0.03 min round |  0.10 min total |  32868.77 day sales |
Predict | Day: 4
##########  0.03 min round |  0.14 min total |  32818.96 day sales |
Predict | Day: 5
##########  0.03 min round |  0.17 min total |  36712.08 day sales |
Predict | Day: 6
##########  0.03 min round |  0.20 min total |  45408.91 day sales |
Predict | Day: 7
##########  0.03 min round |  0.24 min total |  46059.21 day sales |
Predict | Day: 8
##########  0.03 min round |  0.27 min total |  38449.97 day sales |
Predict | Day: 9
##########  0.03 min round |  0.30 min total |  33325.46 day sales |
Predict | Day: 10
##########  0.03 min round |  0.34 min total |  36173.20 day sales |
Predict | Day: 11
##########  0.03 min round |  0.37 min total |  34918.86 day sales |
Predict | Day: 12
##########  0.03 min round |  0.41

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.916198,0.855755,0.855087,0.862295,0.929817,1.027060,1.002394,0.996882,0.908003,...,0.976587,1.041816,1.026873,0.900835,0.856852,0.853284,0.859455,0.956070,1.051159,0.985147
1,HOBBIES_1_002_CA_1_evaluation,0.259378,0.198936,0.198267,0.205475,0.291093,0.393497,0.368831,0.320767,0.255438,...,0.312371,0.382761,0.367818,0.261054,0.217072,0.210945,0.221760,0.308236,0.408486,0.303568
2,HOBBIES_1_003_CA_1_evaluation,0.352444,0.292002,0.291333,0.298541,0.386939,0.489308,0.464643,0.437383,0.348504,...,0.414378,0.484732,0.469789,0.352050,0.308067,0.301941,0.312755,0.402011,0.502226,0.381063
3,HOBBIES_1_004_CA_1_evaluation,1.614464,1.516227,1.515558,1.522767,2.102348,3.197759,3.173093,1.677143,1.568475,...,2.159337,3.222733,3.207790,1.604962,1.523185,1.484308,1.490479,2.070918,3.164175,2.961910
4,HOBBIES_1_005_CA_1_evaluation,0.925900,0.859456,0.858787,0.865995,0.943679,1.270056,1.288160,1.013517,0.918637,...,0.976173,1.270535,1.298361,0.960153,0.910113,0.903987,0.910158,0.986021,1.310243,1.233840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.661350,0.524341,0.524341,0.524341,0.624217,0.984773,0.984773,0.875964,0.524341,...,0.767600,1.040770,1.026265,0.682243,0.565832,0.565832,0.565832,0.665708,1.026265,1.012384
30486,FOODS_3_824_WI_3_evaluation,0.748768,0.611759,0.611759,0.611759,0.770616,1.118942,1.118942,0.986502,0.611759,...,0.913999,1.174939,1.160433,0.769661,0.653251,0.653251,0.653251,0.812108,1.160433,1.160433
30487,FOODS_3_825_WI_3_evaluation,1.064962,0.927953,0.927953,0.927953,1.110474,1.471031,1.471031,1.233322,0.927953,...,1.256857,1.530027,1.515522,1.080623,0.964212,0.964212,0.964212,1.146734,1.507290,1.493409
30488,FOODS_3_826_WI_3_evaluation,1.354253,1.217244,1.217244,1.217244,1.413614,1.679808,1.679808,1.501964,1.217244,...,1.565229,1.744037,1.729531,1.411022,1.294611,1.294611,1.294611,1.490981,1.757176,1.743295


In [15]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL+'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False)